<a href="https://colab.research.google.com/github/dancoster/DrugLab/blob/main/src/Prescription/Prescription_Inputevents_Compare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prescription-Inputevents_Compare

## Import Libraries

In [ ]:
import pandas as pd
import datetime
import numpy as np
from scipy.stats import mannwhitneyu
from scipy import stats
from tqdm import tqdm
import os

## Load Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
PARENT='/content/drive/MyDrive/TAU'
DATA = PARENT+'/Datasets/mimiciii/1.4'

In [ ]:
for i in os.walk(os.path.join(DATA)):
    print(i)

('/content/drive/MyDrive/TAU/Datasets/mimiciii/1.4', [], ['CALLOUT.csv.gz', 'ADMISSIONS.csv.gz', 'CAREGIVERS.csv.gz', 'D_ITEMS.csv.gz', 'D_ICD_PROCEDURES.csv.gz', 'D_ICD_DIAGNOSES.csv.gz', 'ICUSTAYS.csv.gz', 'CHARTEVENTS.csv.gz', 'DRGCODES.csv.gz', 'CPTEVENTS.csv.gz', 'DIAGNOSES_ICD.csv.gz', 'D_CPT.csv.gz', 'D_LABITEMS.csv.gz', 'DATETIMEEVENTS.csv.gz', 'INPUTEVENTS_CV.csv.gz', 'INPUTEVENTS_MV.csv.gz', 'LABEVENTS.csv.gz', 'MICROBIOLOGYEVENTS.csv.gz', 'OUTPUTEVENTS.csv.gz', 'PATIENTS.csv.gz', 'README.md', 'PROCEDURES_ICD.csv.gz', 'PRESCRIPTIONS.csv.gz', 'TRANSFERS.csv.gz', 'PROCEDUREEVENTS_MV.csv.gz', 'SERVICES.csv.gz', 'LICENSE.txt', 'NOTEEVENTS.csv.gz', 'SHA256SUMS.txt', 'index.html'])


## Load Data

### Labevents

In [ ]:
labevents = pd.read_csv(os.path.join(DATA, 'LABEVENTS.csv.gz')).dropna()
d_labitems = pd.read_csv(os.path.join(DATA, 'D_LABITEMS.csv.gz')).dropna()

In [ ]:
labevents.shape, d_labitems.shape

((7881769, 9), (585, 6))

In [ ]:
labevents

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
162,443,3,145834.0,50893,2101-10-20 16:40:00,8.2,8.2,mg/dL,abnormal
163,444,3,145834.0,50902,2101-10-20 16:40:00,99,99.0,mEq/L,abnormal
166,447,3,145834.0,50912,2101-10-20 16:40:00,3.2,3.2,mg/dL,abnormal
169,450,3,145834.0,50970,2101-10-20 16:40:00,4.8,4.8,mg/dL,abnormal
170,451,3,145834.0,50971,2101-10-20 16:40:00,5.4,5.4,mEq/L,abnormal
...,...,...,...,...,...,...,...,...,...
27854045,27428430,96443,103219.0,50862,2109-12-30 01:40:00,2.2,2.2,g/dL,abnormal
27854046,27428431,96443,103219.0,50863,2109-12-30 01:40:00,172,172.0,IU/L,abnormal
27854049,27428434,96443,103219.0,50878,2109-12-30 01:40:00,467,467.0,IU/L,abnormal
27854051,27428436,96443,103219.0,50885,2109-12-30 01:40:00,2.1,2.1,mg/dL,abnormal


In [ ]:
labValues = pd.merge(labevents, d_labitems, on='ITEMID', how='inner')

In [ ]:
labValues.shape

(7876570, 14)

In [ ]:
# subject_id,l.hadm_id, d.label, l.valuenum, l.valueuom, l.charttime
labValues = labValues[['SUBJECT_ID', 'HADM_ID', 'LABEL', 'VALUENUM', 'VALUEUOM', 'CHARTTIME']]

In [ ]:
labValues['CHARTTIME'] = pd.to_datetime(labValues['CHARTTIME'],  format='%Y/%m/%d %H:%M:%S')

In [ ]:
labValues

,SUBJECT_ID,HADM_ID,LABEL,VALUENUM,VALUEUOM,CHARTTIME
0,3,145834.0,"Calcium, Total",8.2,mg/dL,2101-10-20 16:40:00
1,3,145834.0,"Calcium, Total",7.6,mg/dL,2101-10-22 04:00:00
2,3,145834.0,"Calcium, Total",7.1,mg/dL,2101-10-22 21:15:00
3,3,145834.0,"Calcium, Total",7.5,mg/dL,2101-10-23 03:45:00
4,3,145834.0,"Calcium, Total",7.4,mg/dL,2101-10-20 19:59:00
...,...,...,...,...,...,...
7876565,53160,138843.0,Plasma,1.0,%,2194-06-27 13:52:00
7876566,96240,168044.0,Plasma,7.0,%,2105-05-16 16:45:00
7876567,40200,108146.0,Sex Hormone Binding Globulin,12.0,nmol/L,2165-01-10 16:00:00
7876568,58010,185994.0,Sex Hormone Binding Globulin,11.0,nmol/L,2109-02-19 00:16:00


In [ ]:
del labevents, d_labitems

### Prescriptions

In [ ]:
prescriptions = pd.read_csv(os.path.join(DATA, 'PRESCRIPTIONS.csv.gz'), nrows=500000).dropna()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
prescriptions.shape

(180684, 19)

In [ ]:
prescriptions.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'STARTDATE', 'ENDDATE',
       'DRUG_TYPE', 'DRUG', 'DRUG_NAME_POE', 'DRUG_NAME_GENERIC',
       'FORMULARY_DRUG_CD', 'GSN', 'NDC', 'PROD_STRENGTH', 'DOSE_VAL_RX',
       'DOSE_UNIT_RX', 'FORM_VAL_DISP', 'FORM_UNIT_DISP', 'ROUTE'],
      dtype='object')

In [ ]:
k = set(prescriptions['SUBJECT_ID'])

In [ ]:
drugPlot = prescriptions['DRUG'].value_counts(sort=True)

### Top 100 Drugs

In [ ]:
top100Drugs = drugPlot.iloc[:100]

In [ ]:
for drug in top100Drugs.iloc[:10].items():
    print(drug)

('Furosemide', 11940)
('Metoprolol', 11502)
('Insulin', 10554)
('Potassium Chloride', 7517)
('Acetaminophen', 5521)
('Magnesium Sulfate', 4861)
('Morphine Sulfate', 4554)
('Lorazepam', 4480)
('Docusate Sodium', 3335)
('Calcium Gluconate', 3121)


### Input Events

In [ ]:
inputevents_mv = pd.read_csv(os.path.join(DATA, 'INPUTEVENTS_MV.csv.gz'), nrows=500000)

In [ ]:
np.array(inputevents_mv['SUBJECT_ID'])

array([27063, 27063, 27063, ..., 98229, 98229, 98229])

In [ ]:
inputevents_mv.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'STARTTIME', 'ENDTIME',
       'ITEMID', 'AMOUNT', 'AMOUNTUOM', 'RATE', 'RATEUOM', 'STORETIME', 'CGID',
       'ORDERID', 'LINKORDERID', 'ORDERCATEGORYNAME',
       'SECONDARYORDERCATEGORYNAME', 'ORDERCOMPONENTTYPEDESCRIPTION',
       'ORDERCATEGORYDESCRIPTION', 'PATIENTWEIGHT', 'TOTALAMOUNT',
       'TOTALAMOUNTUOM', 'ISOPENBAG', 'CONTINUEINNEXTDEPT', 'CANCELREASON',
       'STATUSDESCRIPTION', 'COMMENTS_EDITEDBY', 'COMMENTS_CANCELEDBY',
       'COMMENTS_DATE', 'ORIGINALAMOUNT', 'ORIGINALRATE'],
      dtype='object')

In [ ]:
final_s = list(k.intersection(np.array(inputevents_mv['SUBJECT_ID'])))

In [ ]:
subject = final_s[np.random.randint(0, len(final_s))]

### Item

In [ ]:
d_item = pd.read_csv(os.path.join(DATA, 'D_ITEMS.csv.gz'))

In [ ]:
ditem_inputevents_mv = pd.merge(inputevents_mv, d_item, on='ITEMID', how='inner')

In [ ]:
ditem_inputevents_mv['CATEGORY'].value_counts()

Fluids/Intake              242772
Medications                196463
Antibiotics                 31382
Nutrition - Enteral         18661
Blood Products/Colloids      7929
Nutrition - Parenteral       2790
Nutrition - Supplements         3
Name: CATEGORY, dtype: int64

In [ ]:
ditem_inputevents_mv.columns

Index(['ROW_ID_x', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'STARTTIME',
       'ENDTIME', 'ITEMID', 'AMOUNT', 'AMOUNTUOM', 'RATE', 'RATEUOM',
       'STORETIME', 'CGID', 'ORDERID', 'LINKORDERID', 'ORDERCATEGORYNAME',
       'SECONDARYORDERCATEGORYNAME', 'ORDERCOMPONENTTYPEDESCRIPTION',
       'ORDERCATEGORYDESCRIPTION', 'PATIENTWEIGHT', 'TOTALAMOUNT',
       'TOTALAMOUNTUOM', 'ISOPENBAG', 'CONTINUEINNEXTDEPT', 'CANCELREASON',
       'STATUSDESCRIPTION', 'COMMENTS_EDITEDBY', 'COMMENTS_CANCELEDBY',
       'COMMENTS_DATE', 'ORIGINALAMOUNT', 'ORIGINALRATE', 'ROW_ID_y', 'LABEL',
       'ABBREVIATION', 'DBSOURCE', 'LINKSTO', 'CATEGORY', 'UNITNAME',
       'PARAM_TYPE', 'CONCEPTID'],
      dtype='object')

## Data Analysis

In [ ]:
subject

1578

In [ ]:
s_presc_data = prescriptions[prescriptions['SUBJECT_ID']==subject]
s_presc_data[prescriptions['HADM_ID']==110184].reset_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,index,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,112206,1127297,1578,110184,244588.0,2141-01-02 00:00:00,2141-01-04 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO40,008209,5.107901e+10,40mg Tablet,80,mg,2,TAB,PO
1,112208,1127299,1578,110184,244588.0,2141-01-03 00:00:00,2141-01-04 00:00:00,MAIN,Potassium Chloride,Potassium Chloride,Potassium Chloride,POTA20,022346,5.817702e+10,20 mEq SR Tablet,40,mEq,2,TAB,PO
2,112209,1127300,1578,110184,244588.0,2141-01-03 00:00:00,2141-01-04 00:00:00,MAIN,Naloxone,Naloxone,Naloxone,NALO4I,004510,4.091212e+08,0.4mg/1mL Vial,0.4,mg,1,mL,IV
3,116732,1127268,1578,110184,263049.0,2140-12-23 00:00:00,2140-12-23 00:00:00,MAIN,Morphine Sulfate,Morphine Sulfate,Morphine Sulfate (Syringe),MORP2I,004070,4.091762e+08,2mg Syringe,2-4,mg,1-2,SYR,IV
4,116733,1127271,1578,110184,263049.0,2140-12-23 00:00:00,2140-12-23 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO40I,008205,5.175704e+08,40mg/4mL Vial,80,mg,2,VIAL,IV
5,116734,1127273,1578,110184,263049.0,2140-12-23 00:00:00,2140-12-23 00:00:00,MAIN,Morphine Sulfate,Morphine Sulfate,Morphine Sulfate (Syringe),MORP2I,004070,4.091762e+08,2mg Syringe,2-4,mg,1-2,SYR,IV
6,116735,1127269,1578,110184,263049.0,2140-12-23 00:00:00,2140-12-23 00:00:00,MAIN,Morphine Sulfate,Morphine Sulfate,Morphine Sulfate (Syringe),MORP4I,004072,4.091258e+08,4mg Syringe,2-4,mg,0.5-1,SYR,IV
7,116736,1127270,1578,110184,263049.0,2140-12-23 00:00:00,2140-12-28 00:00:00,MAIN,Oxycodone-Acetaminophen,Oxycodone-Acetaminophen,Oxycodone-Acetaminophen,PERC,004222,6.348106e+10,5mg/325mg Tablet,1-2,TAB,1-2,TAB,PO
8,116737,1127276,1578,110184,263049.0,2140-12-23 00:00:00,2140-12-29 00:00:00,MAIN,Ipratropium Bromide Neb,Ipratropium Bromide Neb,Ipratropium Bromide Neb,IPRA2H,021700,4.879801e+08,2.5mL Vial,1,NEB,1,VIAL,IH
9,116738,1127272,1578,110184,263049.0,2140-12-23 00:00:00,2141-01-03 00:00:00,MAIN,Fentanyl Patch,Fentanyl Patch,Fentanyl Patch *NF*,FENT12P,059102,7.817110e+08,12 mcg/hr Patch,12,mcg/hr,1,PTCH,TP


In [ ]:
s_mv_data = ditem_inputevents_mv[ditem_inputevents_mv['SUBJECT_ID']==subject]
s_mv_data[ditem_inputevents_mv['HADM_ID']==110184].reset_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,index,ROW_ID_x,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,AMOUNT,AMOUNTUOM,...,ORIGINALRATE,ROW_ID_y,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
0,174293,485643,1578,110184,244588.0,2140-12-29 04:00:00,2140-12-29 08:31:00,225158,67.810726,ml,...,15.000000,13758,NaCl 0.9%,NaCl 0.9%,metavision,inputevents_mv,Fluids/Intake,mL,Solution,NaN
1,174294,485645,1578,110184,244588.0,2140-12-30 06:54:00,2140-12-30 08:13:00,225158,9.221176,ml,...,7.000000,13758,NaCl 0.9%,NaCl 0.9%,metavision,inputevents_mv,Fluids/Intake,mL,Solution,NaN
2,174295,485647,1578,110184,244588.0,2140-12-29 10:00:00,2140-12-29 17:45:00,225158,77.500002,ml,...,10.000000,13758,NaCl 0.9%,NaCl 0.9%,metavision,inputevents_mv,Fluids/Intake,mL,Solution,NaN
3,174296,485649,1578,110184,244588.0,2140-12-30 09:57:00,2140-12-30 10:34:00,225158,3.081825,ml,...,5.000000,13758,NaCl 0.9%,NaCl 0.9%,metavision,inputevents_mv,Fluids/Intake,mL,Solution,NaN
4,174297,485653,1578,110184,244588.0,2140-12-29 17:45:00,2140-12-30 03:43:00,225158,100.000000,ml,...,10.033444,13758,NaCl 0.9%,NaCl 0.9%,metavision,inputevents_mv,Fluids/Intake,mL,Solution,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,415479,485677,1578,110184,244588.0,2141-01-01 16:00:00,2141-01-01 16:01:00,225975,1.000000,dose,...,1.000000,14224,Heparin Sodium (Prophylaxis),Heparin Sodium (Prophylaxis),metavision,inputevents_mv,Medications,dose,Solution,NaN
67,415480,485679,1578,110184,244588.0,2140-12-31 23:59:00,2141-01-01 00:00:00,225975,1.000000,dose,...,1.000000,14224,Heparin Sodium (Prophylaxis),Heparin Sodium (Prophylaxis),metavision,inputevents_mv,Medications,dose,Solution,NaN
68,415481,485683,1578,110184,244588.0,2140-12-30 00:19:00,2140-12-30 00:20:00,225975,1.000000,dose,...,1.000000,14224,Heparin Sodium (Prophylaxis),Heparin Sodium (Prophylaxis),metavision,inputevents_mv,Medications,dose,Solution,NaN
69,427830,485630,1578,110184,244588.0,2140-12-31 02:19:00,2140-12-31 22:01:00,225159,197.000004,ml,...,10.000000,13759,NaCl 0.45%,NaCl 0.45%,metavision,inputevents_mv,Fluids/Intake,mL,Solution,NaN
